In [1]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline
import torch

model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")

CODE = "def returnInt() -> <mask>:"
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
outputs

[{'score': 0.2260766625404358,
  'token': 6979,
  'token_str': ' int',
  'sequence': 'def returnInt() -> int:'},
 {'score': 0.157838374376297,
  'token': 5053,
  'token_str': ' Any',
  'sequence': 'def returnInt() -> Any:'},
 {'score': 0.156962051987648,
  'token': 9291,
  'token_str': ' None',
  'sequence': 'def returnInt() -> None:'},
 {'score': 0.1227448433637619,
  'token': 1907,
  'token_str': ' type',
  'sequence': 'def returnInt() -> type:'},
 {'score': 0.06580957025289536,
  'token': 1666,
  'token_str': '...',
  'sequence': 'def returnInt() ->...:'}]

In [2]:
class CustomModel(torch.nn.Module):
    def __init__(self, model, d, vocabulary_size = 50265): 
        super(CustomModel, self).__init__() 
        self.d = d
        self.model = model
        self.config = model.config
        self.layer = torch.nn.Linear(vocabulary_size, d)
    
    def forward(self, input_ids=None, attention_mask=None):
        model_output = self.model.forward(input_ids=input_ids, attention_mask=attention_mask)        
        final_output_tensor = self.layer.forward(model_output[0])
        model_output.logits = final_output_tensor
        return model_output

In [3]:
nl_tokens=tokenizer.tokenize("")
code_tokens=tokenizer.tokenize("def returnInt() -> <mask>")
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
output = model(torch.tensor(tokens_ids)[None,:])[0]
# tokens[7]
output[0][7][torch.argmax(output[0][7])]

tensor(13.4799, grad_fn=<SelectBackward0>)

In [4]:
custom_model = CustomModel(model, 8)
custom_model.forward(torch.tensor(tokens_ids)[None,:])



MaskedLMOutput(loss=None, logits=tensor([[[ 2.7130, -3.9722, -2.3192, -0.2245,  3.0127, -0.3098, -1.4694,
           2.2311],
         [ 2.4862, -3.6400, -2.4464, -0.4541,  3.9962,  0.2308, -1.1025,
           3.6973],
         [-2.5813, -1.6275,  0.1656,  0.2086,  3.4632,  0.7924, -3.4726,
           4.6346],
         [ 6.2069, -6.1644, -1.2828, -0.1582,  3.7551,  0.6661, -3.6167,
           4.1253],
         [ 1.6097, -4.8903,  0.0202, -0.5007,  0.9295, -0.2237, -1.3462,
           3.7941],
         [-2.7313, -2.3866,  1.9490,  1.4005,  0.1838, -0.5778, -2.6306,
           3.0583],
         [ 5.0826, -7.1754, -2.0584,  0.1283,  4.9531, -1.9544, -1.4784,
           2.7675],
         [ 5.8469, -5.7370, -0.9459,  0.1425,  3.0078, -0.6335, -0.7714,
           2.7078],
         [ 2.6567, -4.4579, -0.4027,  1.3881,  2.6523, -0.7998, -1.4220,
           1.7891]]], grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [5]:
custom_model = CustomModel(model, 8)
fill_mask = pipeline('fill-mask', model=custom_model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
outputs

[{'score': 0.4921939969062805,
  'token': 4,
  'token_str': '.',
  'sequence': 'def returnInt() ->.:'},
 {'score': 0.20608124136924744,
  'token': 6,
  'token_str': ',',
  'sequence': 'def returnInt() ->,:'},
 {'score': 0.15533341467380524,
  'token': 1,
  'token_str': '<pad>',
  'sequence': 'def returnInt() ->:'},
 {'score': 0.12704037129878998,
  'token': 7,
  'token_str': ' to',
  'sequence': 'def returnInt() -> to:'},
 {'score': 0.007507771253585815,
  'token': 3,
  'token_str': '<unk>',
  'sequence': 'def returnInt() ->:'}]

In [6]:
class TripletLoss(torch.nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    
    def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)

        return losses.mean()

In [7]:
from tqdm.notebook import tqdm

def tokenize_code(code):
    nl_tokens = tokenizer.tokenize("")
    code_tokens = tokenizer.tokenize(code)
    tokens = [tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    return torch.tensor(tokens_ids)[None,:]

epochs = 1
data = [("def returnInt() -> <mask>:", "def calcInt() -> <mask>:","def returnFloat() -> <mask>:", "int")]

optimizer = torch.optim.Adam(custom_model.parameters(), lr=0.001)
criterion = torch.jit.script(TripletLoss())

for epoch in tqdm(range(epochs), desc="Epochs"):
    custom_model.train()
    running_loss = []
    for step, (t_a, t_p, t_n, anchor_label) in enumerate(data):
        
        optimizer.zero_grad()
        anchor_out = custom_model(tokenize_code(t_a))
        positive_out = custom_model(tokenize_code(t_p))
        negative_out = custom_model(tokenize_code(t_n))
        
        loss = criterion(anchor_out[0], positive_out[0], negative_out[0])
        loss.backward()
        optimizer.step()

Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
fill_mask = pipeline('fill-mask', model=custom_model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
outputs

[{'score': 0.9334127902984619,
  'token': 5,
  'token_str': ' the',
  'sequence': 'def returnInt() -> the:'},
 {'score': 0.06656498461961746,
  'token': 6,
  'token_str': ',',
  'sequence': 'def returnInt() ->,:'},
 {'score': 2.0614741515601054e-05,
  'token': 1,
  'token_str': '<pad>',
  'sequence': 'def returnInt() ->:'},
 {'score': 1.4281600897447788e-06,
  'token': 4,
  'token_str': '.',
  'sequence': 'def returnInt() ->.:'},
 {'score': 1.1090337892483149e-07,
  'token': 3,
  'token_str': '<unk>',
  'sequence': 'def returnInt() ->:'}]

In [9]:
import numpy as np
from annoy import AnnoyIndex
from torch.utils.data import DataLoader
import random
from collections import defaultdict
import time

input_list = [
    "def returnInt() -> <mask>: \n\t x: int = 42 \n\t return x",
    "def setInt(self, x: int) -> <mask>: \n\t self.x = x",
    "def getInt(self) -> <mask>: \n\t return self.x",
    "def concatString(self, s1: str, s2: str) -> <mask>: \n\t return s1 + s2",
    "def setStr(self, s: str) -> <mask>: \n\t self.s = s",
    "def getStr(self) -> <mask>: \n\t return self.s",
    "def isInt(self, x) -> <mask>: \n\t return x % 1 == 0",
    "def doSomething() -> <mask>: \n\t pass",
    "def isString(self, s) -> <mask>: \n\t return type(s, str)"
]

labels = [
    "lol",
    "float",
    "int",
    "str",
    "str",
    "str",
    "bool",
    "None",
    "bool"
]

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
KNN_TREE_SIZE = 20


def create_type_space(inputs=input_list[:4], labels=labels[:4]):
    """
    Creates the type space based on the inputs and theri corresponding labels
    """
    
    # Make sure imputs are labeled
    assert len(inputs) == len(labels)
    
    # Cache the type space mappings
    computed_mapped_batches_train = []
    with torch.no_grad():
        
        # Iterate through the data set
        for inp, label in zip(inputs, labels):
            
            # Tokenize the code
            nl_tokens = tokenizer.tokenize("")
            code_tokens = tokenizer.tokenize(inp)
            tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
            tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
            
            # Get the type space mapping from the model
            output = custom_model.forward(torch.tensor(tokens_ids)[None,:])
            
            # Select masked tokens
            masked_tokens = [c for c, token in enumerate(code_tokens) if token == "<mask>"]
            
            # For this version, assume only one mask
            assert len(masked_tokens) == 1
            
            # Selected only the masked tokens from the output
            vals = output.logits.cpu().numpy()
            predicted_masks = [vals[0][i] for i in masked_tokens]
            
            # Cache the mapping of the masked token only
            computed_mapped_batches_train.append(predicted_masks)
        
        # Create the type space
        annoy_index = create_knn_index(computed_mapped_batches_train, None, computed_mapped_batches_train[0][0].size)
    return annoy_index

def create_knn_index(train_types_embed: np.array, valid_types_embed: np.array, type_embed_dim:int) -> AnnoyIndex:
    """
    Creates KNNs index for given type embedding vectors, taken from Type4Py
    """
    
    annoy_idx = AnnoyIndex(type_embed_dim, 'euclidean')

    for i, v in enumerate(tqdm(train_types_embed, total=len(train_types_embed), desc="KNN index")):
        print(v[0])
        annoy_idx.add_item(i, v[0])

    annoy_idx.build(KNN_TREE_SIZE)
    return annoy_idx

annoy_idx = create_type_space()
print(annoy_idx)

KNN index:   0%|          | 0/4 [00:00<?, ?it/s]

[ -1.6557126   3.7560234   6.8763504   7.6605964   1.8400276  38.817364
   5.5584106 -15.366767 ]
[ -1.2897218  -3.7613978   8.248832    6.8137364   0.7521207  25.91998
  13.156923  -23.85544  ]
[ -2.0800292    1.101558     4.0125074    6.739779     0.67058575
  27.738108     8.450526   -26.016058  ]
[ -0.99758995  -6.3091645    9.213986    13.246502     0.2959993
  28.89913      5.0067225  -25.078497  ]


In [11]:
def map_type(inputs=input_list[4:]):
    """
    Maps an input to the type space
    """
    with torch.no_grad():
        computed_embed_batches_test = []
        computed_embed_labels_test = []
        
        for inp in inputs:
            
            # Tokenize the code
            nl_tokens = tokenizer.tokenize("")
            code_tokens = tokenizer.tokenize(inp)
            tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
            tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
            
            # Get the type space mapping from the model
            output = custom_model.forward(torch.tensor(tokens_ids)[None,:])
            
            # Select masked tokens
            masked_tokens = [c for c, token in enumerate(code_tokens) if token == "<mask>"]
            
            # For this version, assume only one mask
            assert len(masked_tokens) == 1
            
            # Selected only the masked tokens from the output
            vals = output.logits.cpu().numpy()
            predicted_masks = [vals[0][i] for i in masked_tokens]

            # Cache the mapping of the masked token only
            computed_embed_batches_test.append(predicted_masks)
        
        return computed_embed_batches_test

def predict_type(types_embed_array: np.array, types_embed_labels: np.array, indexed_knn: AnnoyIndex, k: int):
    """
    Predict type of given type embedding vectors
    """

    pred_types_embed = []
    pred_types_score = []
    for i, embed_vec in enumerate(tqdm(types_embed_array, total=len(types_embed_array), desc="Finding KNNs & Prediction")):
        
        # Get the distances to the KNN
        idx, dist = indexed_knn.get_nns_by_vector(embed_vec[0], k, include_distances=True)
        
        # Compute the scores according to the formula
        pred_idx_scores = compute_types_score(dist, idx, types_embed_labels)
        
        # Cache the scores and the labels
        pred_types_embed.append([i for (i, s) in pred_idx_scores])
        pred_types_score.append(pred_idx_scores)
    
    return pred_types_embed, pred_types_score

def compute_types_score(types_dist: list, types_idx: list, types_embed_labels: np.array):
        types_dist = 1 / (np.array(types_dist) + 1e-10) ** 2
        types_dist /= np.sum(types_dist)
        types_score = defaultdict(int)
        for n, d in zip(types_idx, types_dist):
            types_score[types_embed_labels[n]] += d
        
        return sorted({t: s for t, s in types_score.items()}.items(), key=lambda kv: kv[1], reverse=True)
    
types_embed_array = map_type()
knn_K = 2
pred_type_embed, pred_type_score = predict_type(types_embed_array, labels[:4], annoy_idx, knn_K,)
print(pred_type_score)

Finding KNNs & Prediction:   0%|          | 0/5 [00:00<?, ?it/s]

[2, 1]
[1, 2]
[2, 1]
[0, 3]
[1, 2]
[[('int', 0.6918132980976004), ('float', 0.30818670190239955)], [('float', 0.7876875761440544), ('int', 0.2123124238559456)], [('int', 0.5149898238188597), ('float', 0.4850101761811402)], [('lol', 0.781909689751271), ('str', 0.21809031024872907)], [('float', 0.6213698540015244), ('int', 0.37863014599847555)]]
